In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
import pandas as pd
# from unet import build_unet
# from metrics import dice_loss, dice_coef

In [2]:
data = pd.read_csv(r"E:\l3\dynamic landslide prediction\Revision Round1\selected_solution\feature_matrix\training_data_cnn_processed.csv")

In [3]:
data["dis2fault"] = (data["dis2fault"]-4.7755097e05) / 5.229909e05
data["GEM_RT475y"] = (data["GEM_RT475y"]-(-4.649508e02)) / 3.875843e03
data["elevation"] = (data["elevation"]-1.581916e02) / 3.958743e03
data["slope"] = (data["slope"]-(-4.542834e02)) / 3.850408e03
data["dis2river"] = (data["dis2river"]-1.357943e04) / 2.763065e04
data["NDVI"] = (data["NDVI"]-2323.23796253) / 8138.420801119
data["pr"] = (data["pr"]-19.02047875) / 32.9010473645

In [4]:
data = data[["dis2fault", "GEM_RT475y", "elevation", "slope", "dis2river", "NDVI", "pr", "lithology", "landcover", "label"]]

In [5]:
data.describe()

,dis2fault,GEM_RT475y,elevation,slope,dis2river,NDVI,pr,lithology,landcover,label
count,475888.000000,475888.000000,475888.000000,475888.000000,475888.000000,475888.000000,475888.000000,475888.000000,475888.000000,475888.000000
mean,-0.054309,0.119982,0.111954,0.119602,-0.100489,0.238054,2.999740,10.321569,10.436502,0.333299
std,0.975630,0.000017,0.175786,0.001961,0.799732,0.311634,7.320184,2.485272,2.799527,0.471393
min,-0.912779,0.119965,-0.040718,0.117983,-0.487153,-0.528909,-91.669437,0.000000,0.000000,0.000000
25%,-0.827685,0.119971,-0.014952,0.118167,-0.429651,-0.027609,-0.146514,10.000000,9.000000,0.000000
50%,-0.341656,0.119976,0.050725,0.118724,-0.330825,0.241052,1.643702,11.000000,10.000000,0.000000
75%,0.296065,0.119989,0.191174,0.120389,-0.085049,0.513683,5.683087,12.000000,12.000000,1.000000
max,4.042389,0.120188,1.280661,0.132667,9.978787,0.862610,29.445249,16.000000,17.000000,1.000000


In [7]:
data_input = np.array(data).T.reshape(10, int(len(data)/49), 7, 7)

In [8]:
data_input = data_input.transpose(1, 2, 3, 0)

In [9]:
data_input.shape

(9712, 7, 7, 10)

In [10]:
X = data_input[:, :, :, :-1]
y = data_input[:, 3, 3, -1]

In [11]:
# Split the feature and target data into training and validation sets using a 80-20 split ratio.
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [12]:
X_train.shape, y_train.shape, X_val.shape, y_val.shape

((5827, 7, 7, 9), (5827,), (1942, 7, 7, 9), (1942,))

In [13]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import (Input, Conv2D, BatchNormalization, Activation, MaxPool2D,
                                     Conv2DTranspose, Concatenate, GlobalAveragePooling2D, Resizing)
from tensorflow.keras.models import Model
import os

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    return x

def encoder_block(inputs, num_filters):
    x = conv_block(inputs, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

def decoder_block(inputs, skip_features, num_filters):
    # 上采样
    x = Conv2DTranspose(num_filters, 2, strides=2, padding="same")(inputs)

    # 获取 skip 的空间尺寸
    skip_shape = tf.keras.backend.int_shape(skip_features)
    target_height, target_width = skip_shape[1], skip_shape[2]

    # Resize x 和 skip_features 到相同尺寸
    x = Resizing(target_height, target_width, interpolation='bilinear')(x)
    skip_resized = Resizing(target_height, target_width, interpolation='bilinear')(skip_features)

    # 拼接并卷积
    x = Concatenate()([skip_resized, x])
    x = conv_block(x, num_filters)
    return x

def compute_max_depth(h, w):
    depth = 0
    while h >= 2 and w >= 2:
        h //= 2
        w //= 2
        depth += 1
    return depth

def build_dynamic_unet(input_shape):
    inputs = Input(input_shape)
    H, W = input_shape[0], input_shape[1]
    max_depth = min(4, compute_max_depth(H, W))  # 最多4层

    encoder_outputs = []
    x = inputs

    filters = [64, 128, 256, 512, 1024]

    # Encoder
    for i in range(max_depth):
        x, p = encoder_block(x, filters[i])
        encoder_outputs.append(x)
        x = p

    # Bottleneck
    x = conv_block(x, filters[max_depth])

    # Decoder
    for i in reversed(range(max_depth)):
        x = decoder_block(x, encoder_outputs[i], filters[i])

    # 输出层
    x = Conv2D(1, 1, padding="same", activation="sigmoid")(x)
    x = GlobalAveragePooling2D()(x)

    model = Model(inputs, x, name="UNet-Auto-NoLambda")
    return model


In [14]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [15]:
# 输入形状为你当前使用的 (7, 7, 9)
input_shape = (7, 7, 9)

""" Seeding """
np.random.seed(42)
tf.random.set_seed(42)

""" Directory for storing files """
create_dir("files")

""" Hyperparameters """
batch_size = 16
lr = 1e-4
num_epochs = 200
model_path = "E:\\phd_l1\\微信公众号\\20250504_UNET\\model.keras"
csv_path = "E:\\phd_l1\\微信公众号\\20250504_UNET\\log.csv"

""" Model """
model = build_dynamic_unet(input_shape)
model.compile(loss="binary_crossentropy", optimizer=Adam(lr), metrics=["accuracy"])

callbacks = [
    ModelCheckpoint(model_path, verbose=1, save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
    CSVLogger(csv_path),
    EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False),
]

model.fit(
    X_train,
    y_train, 
    epochs=num_epochs,
    validation_data=(X_val, y_val),
    callbacks=callbacks
)


Epoch 1/200
183/183 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.8513 - loss: 0.3758
Epoch 1: val_loss improved from inf to 0.35078, saving model to E:\phd_l1\微信公众号\20250504_UNET\model.keras
183/183 ━━━━━━━━━━━━━━━━━━━━ 14s 45ms/step - accuracy: 0.8513 - loss: 0.3756 - val_accuracy: 0.8687 - val_loss: 0.3508 - learning_rate: 1.0000e-04
Epoch 2/200
182/183 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.8671 - loss: 0.3025
Epoch 2: val_loss improved from 0.35078 to 0.29698, saving model to E:\phd_l1\微信公众号\20250504_UNET\model.keras
183/183 ━━━━━━━━━━━━━━━━━━━━ 8s 43ms/step - accuracy: 0.8671 - loss: 0.3024 - val_accuracy: 0.8795 - val_loss: 0.2970 - learning_rate: 1.0000e-04
Epoch 3/200
183/183 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.8764 - loss: 0.2739
Epoch 3: val_loss improved from 0.29698 to 0.29076, saving model to E:\phd_l1\微信公众号\20250504_UNET\model.keras
183/183 ━━━━━━━━━━━━━━━━━━━━ 8s 42ms/step - accuracy: 0.8764 - loss: 0.2739 - val_accuracy: 0.8785 - val_loss: 0.2908

In [16]:
tf.keras.config.enable_unsafe_deserialization()
best_model = tf.keras.models.load_model(model_path)

In [17]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# 预测
y_pred_prob = best_model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)  # 将概率转换为二进制标签

# 计算各项评估指标
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred_prob)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"AUC: {auc:.4f}")

61/61 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step
Accuracy: 0.8739
Precision: 0.8260
Recall: 0.7864
F1-score: 0.8057
AUC: 0.9475
